In [20]:
import os
import pandas as pd
import json

In [21]:
datasets_base_path = "..\\..\\Dataset\\ClusterParsed\\"
info_path = "..\\DatasetSchemaMatch\\"
filename_synonym = "dic_pre_val.txt"
csv_columns_path = ".\\columns\\"
dictionary_path = ".\\dictionary_score\\"
plot_path = ".\\plot\\"
dict_synonyms_path = ".\\synonyms\\"
dict_final_synonyms_path = ".\\final_synonyms\\"
schema_path = ".\\schema\\"

In [26]:
threshold = 0.1
for filename in os.listdir(dictionary_path):

    file_path = os.path.join(dictionary_path, filename)
    cluster = os.path.splitext(filename)[0]

    print("Cluster:", cluster)

    file = open(file_path)
    dictionary_score = json.load(file)
    file.close()

    synonyms = set()
    for main_token, tokens_score in dictionary_score.items():
        for k, v in tokens_score.items():
            if 0 < v < threshold:
                print(main_token, "->", k)


Cluster: cbinsights
01-DDD-city -> 01-DDD-name
01-DDD-city -> 04-iGMM-company
04-iGMM-city -> 01-DDD-name
04-iGMM-city -> 04-iGMM-company
01-DDD-name -> 01-DDD-city
01-DDD-name -> 04-iGMM-city
04-iGMM-company -> 01-DDD-city
04-iGMM-company -> 04-iGMM-city
Cluster: companiesmarketcap
03-gren-price -> 04-iGMM-rank
03-gren-price -> 00-avengers-rank
00-avengers-rank -> 03-gren-price
04-iGMM-rank -> 03-gren-price
Cluster: disfold
00-avengers-ceo -> 08-slytherin-ceo
04-iGMM-ceo -> 08-slytherin-ceo
10-DeBiGa-ceo -> 08-slytherin-ceo
08-slytherin-ceo -> 00-avengers-ceo
08-slytherin-ceo -> 04-iGMM-ceo
08-slytherin-ceo -> 10-DeBiGa-ceo
02-GioPonSpiz-country -> 06-MarScoToc-country
07-silvestri-country -> 06-MarScoToc-country
03-gren-country -> 06-MarScoToc-country
04-iGMM-country -> 06-MarScoToc-country
05-MalPatSaj-country -> 06-MarScoToc-country
10-DeBiGa-country -> 06-MarScoToc-country
06-MarScoToc-country -> 02-GioPonSpiz-country
06-MarScoToc-country -> 07-silvestri-country
06-MarScoToc-count

In [2]:
"ciao".split("-")

['ciao']

In [1]:
import pandas as pd

In [4]:
schema_path = ".\\final_schema\\schema\\idea_final_schema.csv"
df = pd.read_csv(schema_path)

ParserError: Error tokenizing data. C error: Expected 15 fields in line 105021, saw 117530


In [2]:
schema_path = ".\\clusters\\schema\\cbinsights.csv"

In [10]:
df = pd.read_csv(schema_path, low_memory=False)

In [11]:
df

,ID,index,name,country,headquarters,address,founded,employees,market_cap,revenue,profit,industry,sector,categories,ceo;
0,0,0,tcs,NaN,NaN,NaN,1968,NaN,NaN,NaN,NaN,it services & consulting,NaN,NaN,;
1,1,1,accenture,NaN,NaN,NaN,1989,NaN,NaN,NaN,NaN,it services & consulting,NaN,NaN,;
2,2,2,cognizant,NaN,NaN,NaN,1994,NaN,NaN,NaN,NaN,it services & consulting,NaN,NaN,;
3,3,3,icici bank,NaN,NaN,NaN,1994,NaN,NaN,NaN,NaN,banking,NaN,NaN,;
4,4,4,hdfc bank,NaN,NaN,NaN,1994,NaN,NaN,NaN,NaN,banking,NaN,NaN,;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183999,"184591,6389,zensar technologies,,""pune, india""...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184000,"184592,6390,guangzhou zhujiang brewery group,,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184001,184593,6391,zte,NaN,NaN,NaN,1985,NaN,NaN,NaN,NaN,telecommunications equipment networking equipment,NaN,NaN,;
184002,184594,6392,zippo,NaN,bradford,NaN,1932,NaN,NaN,NaN,NaN,manufacturing,NaN,NaN,;
